In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Arquivos utilizados

Os arquivos fornecidos são de:
* correntistas
* correntistas mortos

## Tratando o arquivo de Correntistas

Abrindo o arquivo de correntistas, que está com codificação Windows

In [2]:
bravos=pd.read_csv("correntistas_banco_bravos.csv",sep=";",encoding='windows-1250')

Removendo as colunas que não são de interesse para o problema

In [3]:
bravos.drop(['book1','book2','book3','book4', 'book5', 'popularity', 'isPopular','isNoble', 'numDeadRelations',
             'isMarried', 'Unnamed: 25', 'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
             'mother', 'father', 'heir', 'culture', 'spouse', 'title', 'male']\
            , axis=1\
            , inplace=True)

Alterando o nome da coluna "Capacidade de Pagamento Anual" para "CapPagAnual", para facilitar a manipulação da coluna

In [4]:
bravos.rename (columns={'Capacidade de pagamento anual':'CapPagAnual'}, inplace=True)

Ajustando os campos "Dívida" e "Capacidade de Pagamento Anual" para serem manipulados como números

In [5]:
bravos.Dívida = bravos.Dívida.str.replace('R\$ ','', regex=True).str.replace('.','').str.replace(',','.').astype(float)

In [6]:
bravos.CapPagAnual = bravos.CapPagAnual.str.replace('R\$ ', '', regex=True).str.replace('.','').str.replace(',','.').astype(float)

Criando dicionário de Equivalência de nomes de Casas, corrigindo os nomes que estão errados no arquivo de correntistas

In [7]:
casas_correto = {
                'brotherhood without banners': 'Brotherhood Without Banners',
                'Brotherhood without banners': 'Brotherhood Without Banners',
                'Brotherhood without Banners': 'Brotherhood Without Banners',
                'House Lannnister': 'House Lannister',
                'House Starrkk': 'House Stark',
                }

Utilizando o dicionário acima para arrumar os nomes das casas com a função _replace_

In [8]:
bravos.replace({'house':casas_correto}, inplace=True)

In [9]:
bravos

,name,house,Dívida,CapPagAnual
0,Daenerys Targaryen,House Targaryen,61115.0,1000000.0
1,Tyrion Lannister,House Lannister,12104.0,700000.0
2,Cersei Lannister,House Lannister,54158.0,700000.0
3,Arya Stark,House Stark,9916.0,500000.0
4,Jon Snow,House Stark,86084.0,500000.0
...,...,...,...,...
1942,Matarys Targaryen,House Targaryen,51608.0,70473.0
1943,Daena Targaryen,House Targaryen,32304.0,64028.0
1944,Yormwell,NaN,74909.0,25548.0
1945,Ghael,NaN,79009.0,33253.0


Aqui, removemos os valores nulos das 3 colunas de interesse

In [10]:
bravos.dropna(subset=['name', 'Dívida', 'CapPagAnual', 'house'], inplace=True)

In [11]:
bravos

,name,house,Dívida,CapPagAnual
0,Daenerys Targaryen,House Targaryen,61115.0,1000000.0
1,Tyrion Lannister,House Lannister,12104.0,700000.0
2,Cersei Lannister,House Lannister,54158.0,700000.0
3,Arya Stark,House Stark,9916.0,500000.0
4,Jon Snow,House Stark,86084.0,500000.0
...,...,...,...,...
1939,Tytos Brax,House Brax,8741.0,28388.0
1940,Werlag,House Greyjoy,63881.0,33395.0
1941,Orton Merryweather,House Merryweather,2660.0,73108.0
1942,Matarys Targaryen,House Targaryen,51608.0,70473.0


Precisamos verificar se há nomes duplicados no dataset

In [12]:
duplic = bravos.duplicated()

In [13]:
duplic[duplic == True]

Series([], dtype: bool)

Acima, vemos que não há.
Seguimos, então para analisar o arquivo de óbito

In [14]:
obitos=pd.read_csv("correntistas_obito.csv",sep=";")

Analisando o arquivo de óbitos, vemos que não há nomes repetidos, nem necessidades de maiores tratamentos

In [15]:
obitos

,Name,Allegiances,Gender,Nobility
0,Aegon Frey (Jinglebell),None,1,1
1,Adrack Humble,House Greyjoy,1,1
2,Aemon Targaryen (son of Maekar I),Night's Watch,1,1
3,Aenys Frey,None,0,1
4,Aggar,House Greyjoy,1,0
...,...,...,...,...
176,Squint,House Greyjoy,1,0
177,Squirrel,Wildling,0,0
178,Yezzan Zo Qaggaz,None,1,1
179,Torwynd the Tame,Wildling,1,0


Para podermos fazer o _join_ entre as duas tabelas, precisamos que as colunas "nome" tenha o mesmo título.

In [16]:
obitos.rename (columns={'Name':'name'}, inplace=True)

In [17]:
obitos

,name,Allegiances,Gender,Nobility
0,Aegon Frey (Jinglebell),None,1,1
1,Adrack Humble,House Greyjoy,1,1
2,Aemon Targaryen (son of Maekar I),Night's Watch,1,1
3,Aenys Frey,None,0,1
4,Aggar,House Greyjoy,1,0
...,...,...,...,...
176,Squint,House Greyjoy,1,0
177,Squirrel,Wildling,0,0
178,Yezzan Zo Qaggaz,None,1,1
179,Torwynd the Tame,Wildling,1,0


Para fazermos o _inner join_ (_join_ que lista como resultado apenas os valores que estão em ambas as listas), utilizamos a função _merge_ do pandas

In [18]:
mortos = pd.merge(bravos, obitos, on='name', how='inner')

In [19]:
mortos

,name,house,Dívida,CapPagAnual,Allegiances,Gender,Nobility
0,Maric Seaworth,House Seaworth,27035.0,53588.0,House Baratheon,1,1
1,Rupert Brax,House Brax,53606.0,27307.0,House Lannister,1,0
2,Ryman Frey,House Frey,88702.0,55716.0,None,1,1
3,Mero,Second Sons,48576.0,42815.0,None,1,0
4,Orphan Oss,Night's Watch,29620.0,69996.0,Night's Watch,1,0
...,...,...,...,...,...,...,...
112,Morrec,House Lannister,65827.0,25481.0,House Lannister,1,0
113,Shagwell,Brave Companions,72907.0,26714.0,None,1,0
114,Rickard Karstark,House Karstark,72373.0,74920.0,Stark,1,1
115,Alebelly,House Stark,79433.0,43723.0,House Stark,1,0


Aqui também precisamos verificar se não há nomes duplicados e transformamos o resultado numa lista

In [20]:
nome_de_mortos = mortos.name.unique()

O _dataset_ **bravos_ativo** lista apenas os nomes de correntistas que não estão mortos. A lista de nomes de correntistas mortos vem da lista **nome_de_mortos** gerada acima, com base na lista de correntistas mortos encontrado com o _inner join_.

In [21]:
bravos_ativo = bravos[~bravos.name.isin(nome_de_mortos)]

In [22]:
bravos_ativo

,name,house,Dívida,CapPagAnual
0,Daenerys Targaryen,House Targaryen,61115.0,1000000.0
1,Tyrion Lannister,House Lannister,12104.0,700000.0
2,Cersei Lannister,House Lannister,54158.0,700000.0
3,Arya Stark,House Stark,9916.0,500000.0
4,Jon Snow,House Stark,86084.0,500000.0
...,...,...,...,...
1939,Tytos Brax,House Brax,8741.0,28388.0
1940,Werlag,House Greyjoy,63881.0,33395.0
1941,Orton Merryweather,House Merryweather,2660.0,73108.0
1942,Matarys Targaryen,House Targaryen,51608.0,70473.0


In [23]:
family_agg = bravos_ativo.groupby('house').sum()

In [24]:
family_agg

,Dívida,CapPagAnual
house,,
Alchemists' Guild,404460.0,308600.0
Antler Men,3951.0,64242.0
Band of Nine,88209.0,34549.0
Black Ears,75172.0,50764.0
Blacks,83159.0,187758.0
...,...,...
Three-eyed crow,87184.0,66421.0
Undying Ones,88295.0,30546.0
Unsullied,95428.0,118803.0


**Saldo** indica qual a saúde financeira da família com o Banco de Bravos.

Quanto menor o saldo, maior a dívida do cliente em relação à sua capacidade de pagamento.

Quanto maior o saldo, maior a capacidade de empréstimos daquela família 

In [25]:
family_agg['Saldo'] =  family_agg.apply(lambda row: row.CapPagAnual - row.Dívida, axis = 1)

In [26]:
family_agg = family_agg[family_agg.Saldo >= 0]

In [27]:
def ABC_segmentation(perc):
    '''
    Creates the 3 classes A, B, and C based 
    on quantity percentages (A-60%, B-25%, C-15%)
    '''
    if perc > 0.5:
        return 'A'
    elif perc >= 0.2 and perc <= 0.5:
        return 'B'
    elif perc < 0.2:
        return 'C'

In [28]:
# ordenar por Saldo
family_agg.sort_values(by='Saldo', inplace=True)
# criar a coluna da possibilidade de recebimento acumulado por Familia
family_agg['InvestAcum'] = family_agg['Saldo'].cumsum()
# criar a coluna com o total de investimento possivel
family_agg['TotInvest'] = family_agg['Saldo'].sum()
# criar a coluna do percentual corrente
family_agg['PercCorrente'] =  family_agg['InvestAcum']/family_agg['TotInvest']
# criar a coluna da Classe
family_agg['Classe'] = family_agg['PercCorrente'].apply(ABC_segmentation)

/home/luizangioletti/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/luizangioletti/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/luizangioletti/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

In [29]:
family_agg

,Dívida,CapPagAnual,Saldo,InvestAcum,TotInvest,PercCorrente,Classe
house,,,,,,,
House Turnberry,73105.0,73705.0,600.0,600.0,11896679.0,0.000050,C
House Inchfield,113486.0,114148.0,662.0,1262.0,11896679.0,0.000106,C
House Chester,37027.0,37781.0,754.0,2016.0,11896679.0,0.000169,C
House Stonetree,52492.0,53600.0,1108.0,3124.0,11896679.0,0.000263,C
House Royce of the Gates of the Moon,362737.0,364628.0,1891.0,5015.0,11896679.0,0.000422,C
...,...,...,...,...,...,...,...
House Martell,1035126.0,1321697.0,286571.0,7877900.0,11896679.0,0.662193,A
House Frey,4445069.0,4851390.0,406321.0,8284221.0,11896679.0,0.696347,A
House Stark,3304540.0,4102346.0,797806.0,9082027.0,11896679.0,0.763409,A


In [30]:
family_agg.to_csv('Curva_ABC.csv')

Com os dados acima é possível plotar um gráfico que indica quais casas fazem parte de qual classe de forma que seja possível visualizarmos quais casas devem ser priorizadas na estratégia do próximo ano.
Contudo, não me foi possível (dentro do tempo proposto) aprender a plotar o gráfico para a entrega.